In [1]:
import pandas as pd
import numpy as np
import bqplot
import ipywidgets
import matplotlib.pyplot as plt
from bqplot import market_map
import traitlets
from IPython.display import clear_output
from knapsack_problem_solver import *

In [2]:
trucks = [transit, e350, box_truck, platform]

In [6]:
rest_percent = [0.5] * 3
solver = KnapsackSolver(KnapsackSolverParser(), platform, rest_percent, True)
best_value, best_sol, transformer = solver.solve()
sol_df = transformer.transform_solution(best_sol)
sol_df

C:\Users\zhangyx\Documents\idac\Algorithms\knapsack/../input.csv
3024.8379322976616
2821.15370593892
Knapsack Solved. Average solving time: 2.3216443061828613s (repetitions count 1, number of instances 1)


,GBID,Quantity,Material Class
0,88242142,1,Electrical
1,99417913,69,Electrical
2,22075182,5,Electrical


,GBID,Quantity,Material Class
0,88242142,2,Electrical
1,99417913,69,Electrical
